In [0]:
from pyspark.sql import Row

# escribir algo temporal
spark.createDataFrame([Row(x=1)]).write.mode("overwrite").format("delta").save("/Volumes/ptd_dev/bronze/raw/_tmp_delta")

# listar para validar
display(dbutils.fs.ls("/Volumes/ptd_dev/bronze/raw/_tmp_delta"))

# limpiar
dbutils.fs.rm("/Volumes/ptd_dev/bronze/raw/_tmp_delta", True)

In [0]:
spark.sql("USE CATALOG ptd_dev")
spark.sql("USE SCHEMA bronze")

base = "/Volumes/ptd_dev/bronze/raw/static/"

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

# Carga Universe SP100 (snapshot)
df_uni = (spark.read
          .option("header", True)
          .option("inferSchema", True)
          .csv(f"{base}/universe_sp100_snapshot.csv"))
display(df_uni)
# Normaliza columnas clave
from pyspark.sql.functions import upper, col
df_uni = (df_uni
          .withColumn("ticker", upper(col("ticker")))
          .withColumn("sector", upper(col("sector"))))
df_uni.write.mode("overwrite").saveAsTable("ptd_dev.bronze.universe_sp100_snapshot")

# Carga Ticker Aliases
df_alias = (spark.read
            .option("header", True)
            .option("inferSchema", True)
            .csv(f"{base}/ticker_aliases.csv"))
df_alias = df_alias.withColumn("alias", upper(col("alias"))) \
                   .withColumn("ticker", upper(col("ticker")))
df_alias.write.mode("overwrite").saveAsTable("ptd_dev.bronze.ticker_aliases")

In [0]:
# Quitar duplicados exactos
# Usá la sesión de Spark para leer la tabla como DataFrame
df_alias = spark.table("ptd_dev.bronze.ticker_aliases")
df_alias_clean = df_alias.dropDuplicates(["alias", "ticker"])
df_alias_clean.write.mode("overwrite").saveAsTable("ptd_dev.bronze.ticker_aliases")